In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [2]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
model_name = "csebuetnlp/mT5_m2m_crossSum"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# New section

In [5]:
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
data = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')
data.head()


,title,article,label
0,এই অভিনেতার ‘প্রস্তাব’ মানলে ভারতীয় সিনেমার খো...,স্রেফ দু’টি টুইটেই সোশ্যাল মিডিয়ায় তোলপাড় ফে...,entertainment
1,"সচিন সাংসদ হয়েছেন, তাই চিন্তা কম জেটলির। কেন?",একবার রেল বাজেট। একবার সাধারণ বাজেট। খবরের শি...,national
2,ক্রিকেটার থেকে গাইড! জাতীয় দলের ক্রিকেটারের অব...,কিছুদিন আগেই রাজ্যের গাইড বনে গিয়েছিলেন চেতেশ...,sports
3,"দুষ্কৃতীদের নজরে এটিএম, হাল হকিকত্ ঘুরে দেখলেন...",কলকাতায় দুষ্কৃতীদের নজরে এখন বিভিন্ন ব্যাঙ্কের...,kolkata
4,"সৌজন্যের বালাই নেই, দিনভর আকচা আকচি কং-বিজেপি’র",সারা দিন আকচা আকচিতেই কাটিয়ে দিল দেশের দুই বৃহ...,national


# New Section

In [8]:
X = list(data['article'])

In [9]:
y = list(data['title'])


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Tokenize input and target sequences
input_encodings = tokenizer(X_train, max_length=500, truncation=True, padding=True,return_tensors="pt")
target_encodings = tokenizer(y_train, max_length=128, truncation=True, padding=True,return_tensors="pt")
import torch
# Create PyTorch datasets
train_dataset = torch.utils.data.TensorDataset(input_encodings['input_ids'], input_encodings['attention_mask'],
                                              target_encodings['input_ids'], target_encodings['attention_mask'])


In [12]:
# Tokenize input and target sequences
input_encodings = tokenizer(X_test, max_length=500, truncation=True, padding=True,return_tensors="pt")
target_encodings = tokenizer(y_test, max_length=128, truncation=True, padding=True,return_tensors="pt")

In [13]:
import torch
# Create PyTorch datasets
test_dataset = torch.utils.data.TensorDataset(input_encodings['input_ids'], input_encodings['attention_mask'],
                                              target_encodings['input_ids'], target_encodings['attention_mask'])

# Custom data collation function
def collate_fn(batch):
    input_ids, input_attention_mask, target_ids, target_attention_mask = zip(*batch)
    return {
        'input_ids': torch.stack(input_ids),
        'attention_mask': torch.stack(input_attention_mask),
        'labels': torch.stack(target_ids),
    }


# from transformers import DataCollatorForSeq2Seq

# Create a DataCollatorForSeq2Seq instance
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)





In [14]:
!pip uninstall  accelerate -y
!pip install  accelerate -U
from transformers import TrainingArguments, Trainer

Found existing installation: accelerate 0.22.0
Uninstalling accelerate-0.22.0:
  Successfully uninstalled accelerate-0.22.0
  Using cached accelerate-0.22.0-py3-none-any.whl (251 kB)


In [15]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=2,
)

In [16]:
import torch
# Check if CUDA (GPU support) is available
torch.cuda.empty_cache()
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")
model.to(device)

CUDA is not available. Using CPU.


MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [19]:

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset = test_dataset,
    data_collator=collate_fn
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss
